In [7]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
train_tokens = pd.read_csv('../data/train_x.csv')
train_tags = pd.read_csv('../data/train_y.csv')

In [3]:
train_data = train_tokens.join(train_tags, on="id", how = "inner", rsuffix = "_tag").drop("id_tag",axis=1)

In [4]:
sentences = []
tags = []
temp_tokens = []
temp_tags = []
for row in train_data.iterrows():
    word = row[1]['word']
    tag = row[1]['tag']
    if word!='-DOCSTART-' and word!='.':
        temp_tokens.append(word)
        temp_tags.append(tag)
    if word=='.':
        sentences.append(' '.join(temp_tokens) + ' .')
        tags.append(' '.join(temp_tags) + ' .')
        temp_tokens = []
        temp_tags = []
    

In [8]:
def nGramTagger(n):
    dic = {}
    for line in tags:
        line = line.split(' ')
        line = ['*']*n + line
        for i in range(n,len(line)):
            item = tuple(line[i-n:i])
            if item in dic:
                dic[item]+=1
            else:
                dic[item]=1
    return dic

def wordTagger():
    dic = defaultdict(int) 
    for line1,line2 in zip(sentences,tags):
        for word,tag in zip(line1.split(' '),line2.split(' ')):
            dic[(word,tag)]+=1
    return dic

unigram = nGramTagger(1)
bigram = nGramTagger(2)
trigram = nGramTagger(3)

wordtag = wordTagger()

In [9]:
def get_q(tag_penult,tag_last,tag_current):
    return float(trigram[(tag_penult, tag_last, tag_current)])/float(bigram[(tag_penult, tag_last)])

def get_e(word,tag):
    return float(wordtag[(word,tag)])/float(unigram[tuple([tag])])

In [11]:
len(sentences)

28510

In [42]:
tags

'NNP NNP , CD NNS JJ , MD VB DT NN IN DT JJ NN NNP CD .'

In [38]:
unique_tags = set()
for tag in tags:
    unique_tags = unique_tags.union([t for t in tag.split()])

In [40]:
unique_tags

{'#',
 '$',
 "''",
 '(',
 ')',
 ',',
 '.',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 '``'}

In [41]:
for sentence in sentences:
    tag_penult = '*'
    tag_prev = '*'
    for i in range(len(sentence)):
        q = get_q(tag_penult, tag_prev, tags[i])
        e = get_e(sentence[i], tags[i])

Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
NNP NNP , CD NNS JJ , MD VB DT NN IN DT JJ NN NNP CD .


28510